# calculate $z_{\rm max}$, $\log M_*$, and any additional values

In [1]:
import os
import h5py 
import glob
import numpy as np
from tqdm import tqdm

In [2]:
os.sys.path.append(os.getcwd().replace('/nb', '/bin'))
import svda as SVDA

In [3]:
from astropy import units as U
from astropy.cosmology import Planck13
from scipy.interpolate import interp1d

In [4]:
from provabgs import util as UT
from provabgs import infer as Infer
from provabgs import models as Models

In [5]:
from speclite import filters as specFilter

In [6]:
# -- plotting -- 
import matplotlib as mpl
import matplotlib.pyplot as plt
#mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False


Bad key text.latex.preview in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95 

## read in posteriors for `provabgs` healpix runs and calculate $z_{\rm max}$ for $\theta_{\rm bf}$

In [7]:
dat_dir = '/global/cfs/cdirs/desi/users/chahah/provabgs/svda/'

In [8]:
r_pass = specFilter.load_filters('decam2014-r')

def r_mag(w, f):
    ''' calculate r-band magnitude given w, f
    '''
    flux_z, w_z = r_pass.pad_spectrum(np.atleast_2d(f) * 1e-17*U.erg/U.s/U.cm**2/U.Angstrom, w * U.Angstrom)
    maggies = r_pass.get_ab_maggies(flux_z, wavelength=w_z)
    return 22.5 - 2.5 * np.log10(maggies.as_array()[0][0] * 1e9)


def bgs_faint_color_cut(hpix):
    ''' if True:    rfib < 20.75
        if False:   rfib < 21.5

    also return r-band fiber fraction
    '''
    _x = SVDA.healpix(hpix, target='BGS_FAINT', redux='fuji', survey='sv3')
    coadd = _x[0]

    trans_g = SVDA.mwdust_transmission(coadd['EBV'], 'g',
            np.array(coadd['PHOTSYS']).astype(str),
            match_legacy_surveys=False)
    trans_r = SVDA.mwdust_transmission(coadd['EBV'], 'r',
            np.array(coadd['PHOTSYS']).astype(str),
            match_legacy_surveys=False)
    trans_z = SVDA.mwdust_transmission(coadd['EBV'], 'z',
            np.array(coadd['PHOTSYS']).astype(str),
            match_legacy_surveys=False)
    trans_w = SVDA.mwdust_transmission(coadd['EBV'], 'w1',
            np.array(coadd['PHOTSYS']).astype(str),
            match_legacy_surveys=False)

    g = 22.5 - 2.5*np.log10((coadd['FLUX_G'] / trans_g).clip(1e-16))
    r = 22.5 - 2.5*np.log10((coadd['FLUX_R'] / trans_r).clip(1e-16))
    z = 22.5 - 2.5*np.log10((coadd['FLUX_Z'] / trans_z).clip(1e-16))
    w1 = 22.5 - 2.5*np.log10((coadd['FLUX_W1'] / trans_w).clip(1e-16))

    schlegel_color = (z - w1) - 3/2.5 * (g - r) + 1.2

    return schlegel_color < 0., coadd['FIBERFLUX_R']/coadd['FLUX_R']

In [9]:
m_nmf = Models.NMF(burst=True, emulator=True)
m_nmf._load_emulator_msurv()

input parameters : logmstar, beta1_sfh, beta2_sfh, beta3_sfh, beta4_sfh, fburst, tburst, gamma1_zh, gamma2_zh, dust1, dust2, dust_index


/global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/torch/cuda/__init__.py:145: UserWarning: 
NVIDIA A100-PCIE-40GB with CUDA capability sm_80 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA A100-PCIE-40GB GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


# BGS Bright

In [10]:
hpixs = [int(f.split('-')[3].split('.')[0]) for f in glob.glob(os.path.join(dat_dir, 'provabgs-*.BGS_BRIGHT.hdf5'))]
print('%i healpixels' % len(hpixs))

369 healpixels


In [11]:
for hpix in tqdm(hpixs): 
    #print(f'healpix {hpix}')
    # read posteriors in healpix
    with h5py.File(os.path.join(dat_dir, 'provabgs-sv3-bright-%i.BGS_BRIGHT.hdf5' % hpix), 'r') as fhpix: 
        #if 'zmax' in fpost.keys(): continue 
        if 'redshift' not in fhpix.keys(): 
            print()
            print('%i is problematic' % hpix)
            print()
            continue
        
        zreds   = fhpix['redshift'][...]
        if len(zreds) == 0: continue
    
        targids = fhpix['targetid'][...]
        
        _logp   = fhpix['log_prob'][...]
        _logp   = _logp.reshape((_logp.shape[0], _logp.shape[1] * _logp.shape[2]))

        _theta  = fhpix['samples'][...]
        _theta  = _theta.reshape((_theta.shape[0], _theta.shape[1] * _theta.shape[2], _theta.shape[3]))

        theta_bfs = np.array([tt[imax,:] for imax, tt in zip(np.argmax(_logp, axis=1), _theta)])
    
    # calculate z-max
    ngal = len(zreds)
    zmax = np.zeros(ngal)
    for i in range(ngal): 
        zred, theta_bf = zreds[i], theta_bfs[i] 
        
        z_arr = np.linspace(zred, 0.6, 10)
        dlz = Planck13.luminosity_distance(z_arr).to(U.cm).value
        
        # get best-fit SED 
        w, f = m_nmf.sed(theta_bf[:-1], zred)
        
        w_z = w / (1. + zred) * (1 + z_arr[:,None])
        f_z = f * ((dlz[0]**2 / dlz**2) * (1 + zred)/(1 + z_arr))[:,None]
        
        r_arr = np.array([r_mag(_w, _f) for _w, _f in zip(w_z, f_z)])

        if np.min(r_arr) > 19.5:
            zmax[i] = zred
        elif np.max(r_arr) < 19.5:
            zmax[i] = 0.6
        else:
            fint_rz = interp1d(r_arr, z_arr, kind='cubic')
            zmax[i] = fint_rz(19.5)

    # calculate M* (surviving stellar mass)
    tages = Planck13.age(zreds).value
    logmstar_bf = np.log10(m_nmf._surviving_mass(theta_bfs[:,:12], tages, emulator=True))

    logmstar = []
    for i in range(ngal):
        logmstar.append(np.log10(m_nmf._surviving_mass(_theta[i,:,:-1], tages[i], emulator=True)))
    logmstar = np.array(logmstar) 
    
    # append 
    with h5py.File(os.path.join(dat_dir, f'provabgs-sv3-bright-{hpix}.BGS_BRIGHT.hdf5'), 'r+') as fhpix:
        if 'zmax' in fhpix.keys():
            fhpix['zmax'][...] = zmax
        else:
            fhpix.create_dataset('zmax', data=zmax)

        if 'logMstar' in fhpix.keys():
            fhpix['logMstar'][...] = logmstar
        else:
            fhpix.create_dataset('logMstar', data=logmstar)

        if 'theta_bf' in fhpix.keys():
            fhpix['theta_bf'][...] = theta_bfs
        else:
            fhpix.create_dataset('theta_bf', data=theta_bfs)

        if 'logMstar_bf' in fhpix.keys():
            fhpix['logMstar_bf'][...] = logmstar_bf
        else:
            fhpix.create_dataset('logMstar_bf', data=logmstar_bf)

 54%|█████▎    | 198/369 [1:18:22<1:21:18, 28.53s/it]


9932 is problematic



100%|██████████| 369/369 [2:19:02<00:00, 22.61s/it]  


# BGS Faint

In [10]:
hpixs = [int(f.split('-')[3].split('.')[0]) for f in glob.glob(os.path.join(dat_dir, 'provabgs-*.BGS_FAINT.hdf5'))]
print('%i healpixels' % len(hpixs))

360 healpixels


In [11]:
for hpix in tqdm(hpixs): 
    # read posteriors in healpix
    with h5py.File(os.path.join(dat_dir, 'provabgs-sv3-bright-%i.BGS_FAINT.hdf5' % hpix), 'r') as fhpix: 
        #if 'zmax' in fpost.keys(): continue 
        if 'redshift' not in fhpix.keys(): 
            print()
            print('%i is problematic' % hpix)
            print()
            continue
        
        zreds   = fhpix['redshift'][...]
        if len(zreds) == 0: continue
    
        targids = fhpix['targetid'][...]
        
        _logp   = fhpix['log_prob'][...]
        _logp   = _logp.reshape((_logp.shape[0], _logp.shape[1] * _logp.shape[2]))

        _theta  = fhpix['samples'][...]
        _theta  = _theta.reshape((_theta.shape[0], _theta.shape[1] * _theta.shape[2], _theta.shape[3]))

        theta_bfs = np.array([tt[imax,:] for imax, tt in zip(np.argmax(_logp, axis=1), _theta)])

    # BGS Faint color cut and fiber aperture fraction
    faint_color_cut, f_fiber = bgs_faint_color_cut(hpix)
    
    # calculate z-max
    ngal = len(zreds)
    zmax = np.zeros(ngal)
    for i in range(ngal): 
        zred, theta_bf = zreds[i], theta_bfs[i] 
        
        z_arr = np.linspace(zred, 0.6, 10)
        dlz = Planck13.luminosity_distance(z_arr).to(U.cm).value
        
        # get best-fit SED 
        w, f = m_nmf.sed(theta_bf[:-1], zred)
        
        w_z = w / (1. + zred) * (1 + z_arr[:,None])
        f_z = f * ((dlz[0]**2 / dlz**2) * (1 + zred)/(1 + z_arr))[:,None]
        
        r_arr = np.array([r_mag(_w, _f) for _w, _f in zip(w_z, f_z)])
        
        r_fib = r_arr - 2.5 * np.log10(f_fiber[i])
        r_fib_cut = [21.5, 20.75][faint_color_cut[i]]

        if np.min(r_fib) > r_fib_cut:
            zmax[i] = zred
        elif np.max(r_fib) < r_fib_cut:
            zmax[i] = 0.6
        else:
            fint_rz = interp1d(r_fib, z_arr, kind='cubic')
            zmax[i] = fint_rz(r_fib_cut)

    # calculate M* (surviving stellar mass)
    tages = Planck13.age(zreds).value
    logmstar_bf = np.log10(m_nmf._surviving_mass(theta_bfs[:,:12], tages, emulator=True))

    logmstar = []
    for i in range(ngal):
        logmstar.append(np.log10(m_nmf._surviving_mass(_theta[i,:,:-1], tages[i], emulator=True)))
    logmstar = np.array(logmstar) 
    
    # append 
    with h5py.File(os.path.join(dat_dir, f'provabgs-sv3-bright-{hpix}.BGS_FAINT.hdf5'), 'r+') as fhpix:
        if 'zmax' in fhpix.keys():
            fhpix['zmax'][...] = zmax
        else:
            fhpix.create_dataset('zmax', data=zmax)

        if 'logMstar' in fhpix.keys():
            fhpix['logMstar'][...] = logmstar
        else:
            fhpix.create_dataset('logMstar', data=logmstar)

        if 'theta_bf' in fhpix.keys():
            fhpix['theta_bf'][...] = theta_bfs
        else:
            fhpix.create_dataset('theta_bf', data=theta_bfs)

        if 'logMstar_bf' in fhpix.keys():
            fhpix['logMstar_bf'][...] = logmstar_bf
        else:
            fhpix.create_dataset('logMstar_bf', data=logmstar_bf)

100%|██████████| 360/360 [1:42:26<00:00, 17.07s/it]
